In [1]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from os import listdir
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.applications import vgg16, resnet50

In [2]:
def add_layers(bottom_model):
    top_model = bottom_model.output
    top_model = GlobalAveragePooling2D()(top_model)
    top_model = Dense(1024,activation='relu')(top_model)
    top_model = Dense(512,activation='relu')(top_model)
    top_model = Dense(3, activation='softmax')(top_model)
    return top_model

transfer learning vgg16, resnet50, inception

In [3]:
model = tf.keras.applications.ResNet50(weights = 'imagenet',
                    include_top = False,
                    input_shape= (224,224,3))

In [4]:
for layer in model.layers:
    layer.trainable = True

In [5]:
model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [6]:
# model = tf.keras.models.Sequential([
#     tf.keras.layers.Conv2D(100, (3,3), activation='relu', input_shape=(150, 150, 3)),
#     tf.keras.layers.MaxPooling2D(2,2),
    
#     tf.keras.layers.Conv2D(100, (3,3), activation='relu'),
#     tf.keras.layers.MaxPooling2D(2,2),
    
#     tf.keras.layers.Flatten(),
#     tf.keras.layers.Dropout(0.5),
#     tf.keras.layers.Dense(50, activation='relu'),
#     tf.keras.layers.Dense(3, activation='softmax')
# ])
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

In [7]:
FC_Head = add_layers(model)

model = Model(inputs = model.input, outputs = FC_Head)

In [8]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
______________________________________________________________________________________________

In [9]:
training_dir = 'split/train'

train_datagen = ImageDataGenerator(rescale=1.0/255,
                                   rotation_range = 40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(training_dir,
                                                    batch_size=25,
                                                    target_size=(224,224))

test_dir = 'split/test'

test_datagen = ImageDataGenerator(rescale=1.0/255)

test_generator = test_datagen.flow_from_directory(test_dir,
                                                  batch_size=25,
                                                  target_size=(224,224))

checkpoint = ModelCheckpoint('model-{epoch:03d}.model',
                             monitor='val_loss',
                             verbose=0,save_best_only=True,
                             mode='auto')

earlystop = EarlyStopping(monitor = 'val_loss',
                          min_delta=0,
                          patience=10,
                          restore_best_weights = True)

Found 130046 images belonging to 3 classes.
Found 37160 images belonging to 3 classes.


In [10]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

In [ ]:
history = model.fit(train_generator,
                             epochs=30,
                             validation_data=test_generator,
                             callbacks=[checkpoint, earlystop])

Epoch 1/30
5202/5202 [==============================] - 3154s 605ms/step - loss: 0.0535 - acc: 0.9841 - val_loss: 0.0251 - val_acc: 0.9930
INFO:tensorflow:Assets written to: model-001.model\assets


D:\Anaconda\envs\tf2\lib\site-packages\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


Epoch 2/30
5202/5202 [==============================] - 3189s 613ms/step - loss: 0.0251 - acc: 0.9928 - val_loss: 0.0294 - val_acc: 0.9897
Epoch 3/30
5202/5202 [==============================] - 3202s 616ms/step - loss: 0.0199 - acc: 0.9943 - val_loss: 0.0165 - val_acc: 0.9965
INFO:tensorflow:Assets written to: model-003.model\assets


D:\Anaconda\envs\tf2\lib\site-packages\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


Epoch 4/30
5202/5202 [==============================] - 3180s 611ms/step - loss: 0.0224 - acc: 0.9942 - val_loss: 0.3892 - val_acc: 0.9083
Epoch 5/30
5202/5202 [==============================] - 3176s 610ms/step - loss: 0.0136 - acc: 0.9965 - val_loss: 0.0101 - val_acc: 0.9968
INFO:tensorflow:Assets written to: model-005.model\assets


...

Epoch 8/30
5202/5202 [==============================] - 3362s 646ms/step - loss: 0.0098 - acc: 0.9974 - val_loss: 0.0036 - val_acc: 0.9988
Epoch 9/30
5202/5202 [==============================] - 3318s 638ms/step - loss: 0.0105 - acc: 0.9973 - val_loss: 0.0021 - val_acc: 0.9994
INFO:tensorflow:Assets written to: model-009.model\assets


D:\Anaconda\envs\tf2\lib\site-packages\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


Epoch 10/30
5202/5202 [==============================] - 3343s 643ms/step - loss: 0.0089 - acc: 0.9978 - val_loss: 0.0043 - val_acc: 0.9988
Epoch 11/30
5202/5202 [==============================] - 3368s 647ms/step - loss: 0.0076 - acc: 0.9980 - val_loss: 0.0254 - val_acc: 0.9955
Epoch 12/30
5202/5202 [==============================] - 3361s 646ms/step - loss: 0.0076 - acc: 0.9982 - val_loss: 0.0038 - val_acc: 0.9990
Epoch 13/30
5202/5202 [==============================] - 3525s 678ms/step - loss: 0.0071 - acc: 0.9984 - val_loss: 0.0257 - val_acc: 0.9944
Epoch 14/30
5202/5202 [==============================] - 3362s 646ms/step - loss: 0.0076 - acc: 0.9983 - val_loss: 2.4471 - val_acc: 0.6597
Epoch 15/30
5202/5202 [==============================] - 3345s 643ms/step - loss: 0.0064 - acc: 0.9985 - val_loss: 0.0019 - val_acc: 0.9997
INFO:tensorflow:Assets written to: model-015.model\assets


D:\Anaconda\envs\tf2\lib\site-packages\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


Epoch 16/30
5202/5202 [==============================] - 3370s 648ms/step - loss: 0.0065 - acc: 0.9985 - val_loss: 0.0065 - val_acc: 0.9983
Epoch 17/30
5202/5202 [==============================] - 3401s 654ms/step - loss: 0.0063 - acc: 0.9985 - val_loss: 0.0972 - val_acc: 0.9715
Epoch 18/30
5202/5202 [==============================] - 3358s 646ms/step - loss: 0.0057 - acc: 0.9986 - val_loss: 0.0031 - val_acc: 0.9991
Epoch 19/30
5202/5202 [==============================] - 3369s 648ms/step - loss: 0.0054 - acc: 0.9988 - val_loss: 103.2029 - val_acc: 0.8639
Epoch 20/30
5202/5202 [==============================] - 3365s 647ms/step - loss: 0.0136 - acc: 0.9982 - val_loss: 0.0023 - val_acc: 0.9995
Epoch 21/30
5202/5202 [==============================] - 3527s 678ms/step - loss: 0.0050 - acc: 0.9989 - val_loss: 0.0371 - val_acc: 0.9935
Epoch 22/30
5202/5202 [==============================] - 3576s 687ms/step - loss: 0.0059 - acc: 0.9987 - val_loss: 0.0025 - val_acc: 0.9995
Epoch 23/30
5202/5

In [ ]:
print(history.history.keys())

In [ ]:
## Summarize accuracy

plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train','Test'], loc='upper left')
plt.savefig('model_acc.png', dpi=200, format='png')

In [ ]:
## Summarize loss

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train','Test'], loc='upper right')
plt.savefig('model_loss.png', dpi=200, format='png')

In [ ]:
model = load_model('model-005.model')

In [ ]:
val_dir = 'split/val'

val_datagen = ImageDataGenerator(rescale=1.0/255)

val_generator = val_datagen.flow_from_directory(val_dir,
                                                batch_size=25,
                                                target_size=(224,224))

In [ ]:
model.evaluate(val_generator)